In [ ]:
from __future__ import print_function
import ex1_kwstest as kws 
import ex2_getVoice2Text as gv2t 
import ex4_getText2VoiceStream as tts 
import MicrophoneStream as MS 
import RPi.GPIO as GPIO 
import time

import argparse, pafy, ffmpeg, pyaudio

from googleapiclient.discovery import build 
from googleapiclient.errors import HttpError

DEVELOPER_KEY = '사용자의 API 키 입력' 
YOUTUBE_API_SERVICE_NAME = 'youtube' 
YOUTUBE_API_VERSION = 'v3'

GPIO.cleanup()
GPIO.setmode(GPIO.BOARD) 
GPIO.setwarnings(False) 
GPIO.setup(29, GPIO.IN, pull_up_down=GPIO.PUD_UP) 
GPIO.setup(31, GPIO.OUT) btn_status = False
play_flag = 0
def callback(channel):    
    print("falling edge detected from pin {}".format(channel))
    global btn_status  
    btn_status = True  
    print(btn_status)  
    global play_flag  play_flag = 1

GPIO.add_event_detect(29, GPIO.FALLING, callback=callback, bouncetime=10)

def youtube_search(options):  
    try:    
        youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
    parser = argparse.ArgumentParser()    
    parser.add_argument('--q', help='Search term', default=options)    
    parser.add_argument('--max-results', help='Max results', default=25)    
    args = parser.parse_args()
    
    search_response = youtube.search().list(      
        q=args.q,      
        part='id,snippet',      
        maxResults=args.max_results    
    ).execute()
    
    videos = []    
    url = []
    
    for search_result in search_response.get('items', []):      
        if search_result['id']['kind'] == 'youtube#video':        
            videos.append('%s (%s)' % (search_result['snippet']['title'],search_result['id']['videoId']))        
            url.append(search_result['id']['videoId'])
            
    resultURL = "https://www.youtube.com/watch?v=" + url[0]    
    return resultURL

  except :    
        print("Youtube Error")

        
def play_with_url(play_url):  
    print(play_url)  
    video = pafy.new(play_url)  
    best = video.getbestaudio()  
    playurl = best.url  
    global play_flag  
    play_flag = 0
    
    pya = pyaudio.PyAudio()  
    stream = pya.open(format=pya.get_format_from_width(width=2), channels=1, rate=16000,                    
                      output=True)
    try:      
        process = (ffmpeg          
                   .input(playurl, err_detect='ignore_err', reconnect=1, reconnect_streamed=1,                 
                          reconnect_delay_max=5)          
                   .output('pipe:', format='wav', audio_bitrate=16000, ab=64, ac=1, ar='16k')          
                   .overwrite_output()          
                   .run_async(pipe_stdout=True)      
        )
        while True:        
            if play_flag == 0 :          
                in_bytes = process.stdout.read(4096)          
                if not in_bytes:              
                    break          
                stream.write(in_bytes)        
            else:          
                break  
        finally:      
            stream.stop_stream()      
            stream.close()

In [ ]:
import random

'''
solutionList_env = ["창문을 열어 환기를 시켜주세요",
                 "제습기를 사용해보시는건 어떨까요?",
                 "난방을 틀어보세요",
                 "가습기를 사용해보세요",
                 "물이 젖은 수건을 실내에 걸어두시면 습도를 올리는데 도움이 됩니다.",
                 "환풍기를 사용해보세요",
                 "선풍기를 사용해보세요"
                 "커튼을 쳐보세요"]

solutionList_per = ["따뜻한 물을 마셔보세요",
                 "차가운 물을 마셔보세요",
                 "내복을 입어 체온을 따뜻하게 유지시켜주세요",
                 "목에 손수건을 둘러보세요",
                 "따뜻한 물로 샤워해보시는건 어떨까요?" ]
'''

down_t = ["창문을 열어 환기를 시켜주세요",
          "선풍기를 집 밖을 향해서 틀어주세요",
          "커튼을 쳐주세요",
          "물을 마셔주세요",
          "따뜻한 물로 샤워해주세요"]  # 3 

down_h = ["창문을 열어 환기를 시켜주세요",
          "제습기를 틀어주세요",
          "환풍기를 틀어주세요",
          "물을 마셔주세요"] # 3

down_t_dust = ["선풍기를 집 밖을 향해서 틀어주세요",
               "커튼을 쳐주세요",
               "물을 마셔주세요",
               "따뜻한 물로 샤워해주세요"] # 2


down_h_dust = ["제습기를 틀어주세요",
              "환풍기를 틀어주세요",
              "물을 마셔주세요"]   # 2

up_t = ["난방을 틀어주세요.",
        "따뜻한 물로 샤워해주세요.",
        "따뜻한 물을 마셔주세요.",
        "내복을 입어주세요.",
        "양말을 신어주세요.",  
        "목에 손수건을 둘러주세요." ]   # 1

up_h = ["가습기를 틀어주세요",
        "빨래나 물에 젖은 손수건을 널어주세요",
        "목에 손수건이 스카프를 둘러주세요,
        "물을 마셔주세요" ]   # 1


season == ''
today_t = 0.0
today_h = 0.0
find_dust = 0.0
t = 0.0
h = 0.0
ans = ""


def giveSolution():

    if season == '여름':
        if today_t - t >= 8:
            print("현재 밖과의 온도차가 매우 큽니다.")
            print("에어컨이나 선풍기의 전원을 꺼주세요.")
        else:
            if t >= 30 and h >= 65:
                print("현재 온도는 높고 습도도 높습니다.")

                if find_dust >= 75 or today_h > h :
                    ans = down_t_dust[random.randrange(0,2)] + '그리고' + down_h_dust[random.randrange(0,2)]
                else:
                    ans = down_t[random.randrange(0,3)] + '그리고' + down_h[random.randrange(0,3)]

            elif t >= 30 and h <= 35:
                print("현재 온도는 높고 습도는 낮습니다.")
                ans = down_t[random.randrange(0,3)] + '그리고' + up_h[random.randrange(0,1)]
                
            elif t >= 30 and 35 < h < 65:
                print("현재 온도가 높습니다. ")
                 ans = down_t[random.randrange(0,3)]
                 
            elif t < 30 and h >= 65:
                print("현재 습도가 높습니다. ")
                if find_dust >= 75 or today_h > h :
                    ans = down_h_dust[random.randrange(0,2)]
                else:
                    ans = down_h[random.randrange(0,3)]

            elif t < 30 and h <= 35:
                print("현재 습도가 낮습니다. ")
                ans = up_h[random.randrange(0,1)]
                
            else:
                print("현재 온도와 습도가 모두 쾌적한 상태입니다, ")

    elif season == '겨울':
        if t <= 22 and h >= 65:
            print("현재 온도는 낮고 습도는 높습니다.")
            if find_dust >= 75 or today_h > h :
                ans = up_t[random.randrange(0,1)] + '그리고' + down_h_dust[random.randrange(0,2)]
            else:
                ans = up_t[random.randrange(0,1)] + '그리고' + down_h[random.randrange(0,3)]

        elif t <= 22 and h <= 35:
            print("현재 온도는 낮고 습도도 낮습니다.")
            ans = up_t[random.randrange(0,1)] + '그리고' + up_h[random.randrange(0,1)]

        elif t <= 22 and 35<h<65:
            print("현재 온도가 낮습니다.")
            ans = up_t[random.randrange(0,1)]

        elif t > 22 and h >= 65:
            print("현재 습도가 높습니다.")
            if find_dust >= 75 or today_h > h :
                ans = down_h_dust[random.randrange(0,2)]
            else:
                ans = down_h[random.randrange(0,3)]

        elif t > 22 and h <= 35:
            print("현재 습도가 낮습니다. "
            ans = up_h[random.randrange(0,1)]

        else:
            print("온도와 습도가 모두 쾌적한 상태입니다.")

    else:
        if today_t - t >= 8:
            print("현재 밖과의 온도 차이가 매우 큽니다.")
            print("에어컨이나 선풍기의 전원을 꺼주세요.")

        else:
            if 29 <= t and h >= 65:
                print("현재 온도가 높고 습도도 높습니다. ")
                if find_dust >= 75 or today_h > h :
                    ans = down_t_dust[random.randrange(0,2)] + '그리고' + down_h_dust[random.randrange(0,2)]
                else:
                    ans = down_t[random.randrange(0,3)] + '그리고' + down_h[random.randrange(0,3)]

            elif 29 <= t and h <= 35:
                print("현재 온도가 높고 습도는 낮습니다.")
                ans = down_t[random.randrange(0,3)] + '그리고' + up_h[random.randrange(0,1)]

            elif 22 >= t and h >= 65:
                print("현재 온도는 낮고 습도는 높습니다.")
                if find_dust >= 75 or today_h > h :
                    ans = up_t[random.randrange(0,1)] + '그리고' + down_h_dust[random.randrange(0,2)]
                else:
                    ans = up_t[random.randrange(0,1)] + '그리고' + down_h[random.randrange(0,3)]

            elif 22 >= t and h <= 35:
                print("현재 온도는 낮고 습도도 낮습니다.")
                ans = up_t[random.randrange(0,1)] + '그리고' + up_h[random.randrange(0,1)]

            elif 29 <= t and 35 < h < 65 :
                print("현재 온도가 높습니다.")
                ans = down_t[random.randrange(0,3)]

            elif 22 >= t and 35 < h < 65 :
                print("현재 온도가 낮습니다.")
                ans = up_t[random.randrange(0,1)]

            elif 22 < t < 29 and h >= 65 :
                print("현재 습도가 높습니다.")
                if find_dust >= 75 or today_h > h :
                    ans = down_h_dust[random.randrange(0,2)]
                else:
                    ans = down_h[random.randrange(0,3)]

            elif 22 < t < 29 and h <= 35 :
                print("현재 습도가 낮습니다.")
                ans = up_h[random.randrange(0,1)]

            else:
                print("현재 온도와 습도가 모두 쾌적한 상태입니다,")